In [ ]:
# CAGED (Cadastro Geral de Empregados e Desempregados)
# name:         caged_xlsx_to_csv
# date:         29/03/2022 - 01/04/2022
# description:  caged xlsx file data processed, optimized and saved into csv file

import pandas as pd


In [ ]:
# creates panda database with all file sheets               - dfb
# creates panda dataframe with specific sheets              - first_df
# creates /path/files names for getting and saving data     - file_in, file_out

file_in = '3-tabelas_MAR.xlsx'
file_out = 'caged.csv'

skiprows = 4  # [0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0]

dfb = pd.read_excel(file_in, sheet_name=None, skiprows=skiprows)
first_df = dfb['Tabela 5.1']
# df


In [ ]:
# optimizes data mensal evolution by dates with Saldos

months_names_br = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
                   'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')
month_names_dec = ('01',      '02',        '03',    '04',    '05',
                   '06',    '07',    '08',     '09',       '10',      '11',       '12')

# prepare dict variable to the replacements
mbr2mdec = dict(zip(months_names_br, month_names_dec))

df2 = first_df.copy()

# selecting date cells only
df2 = df2[(df2['Mês'].str.slice(-4) >= '1900') &
          (df2['Mês'].str.slice(-4) <= '2099')]

# months name to months numbers replacements
df2['Mês'] = df2['Mês'].replace(mbr2mdec, regex=True)
df2['Mês'] = df2['Mês'].str.replace('/', '-')
df2['Mês'] = pd.to_datetime(
    df2['Mês'], format='%Y-%m-%d', infer_datetime_format=True)


def try_int(val):
    try:
        val = float(val)
        val = int(val)
        return True
    except:
        return False


filter = df2['Saldos'].apply(try_int)
df2.drop(df2[filter == False].index, inplace=True)

df2['Saldos'] = df2['Saldos'].astype(int)
first_out_df = df2[['Mês', 'Saldos']]
first_out_df


In [ ]:
# creates panda dataframe with specific sheets  - second_df


second_df = pd.read_excel(file_in, sheet_name=11, skiprows=4, header=[0, 1])


In [ ]:
# optimizes data mensal evolution by dates and regions with Saldos

regions_table = ('Região e UF', 'Unnamed: 1_level_1')
regions_names = ('Norte', 'Nordeste', 'Sudeste',
                 'Sul', 'Centro-Oeste', 'Não identificado')

# copying df for reutilizing
df2 = second_df.copy()


# function to create a dataframe columns list to use
def createColumnsList(param_df):
    columns_list = []
    column_names = param_df.columns.values.tolist()
    for column_name in column_names:
        # column_name has 2 parts (date, info_type)
        col_date, col_info = column_name
        month = col_date.split('/')[0]
        if month in months_names_br:
            if col_info.lower().find('saldos') != -1:
                columns_list.append(column_name)

    return(columns_list)


# function to clean Saldos from invalid values
def cleanList(cl_df2, list_to_clean):
    for month in list_to_clean:
        print('-MES----------------------------------------------------')
        print(month)
        print('-FILTRO----------------------------------------------------')
        filter = cl_df2[month].apply(try_int)
        print(filter)
        df_tmp = cl_df2[month]
        df_tmp.transpose
        print('-COLUNA_SALDO----------------------------------------------')
        print(df_tmp)
        print('-VALORES_INVALIDOS-----------------------------------------')
        print(df_tmp[filter == False])
        print('-COLUNA ARRUMADA-------------------------------------------')
        cl_df2.drop(df_tmp[filter == False].index, inplace=True)
        print(cl_df2[[month]])

        return(cl_df2)


months_list = createColumnsList(df2)
df2 = cleanList(df2, months_list)

# selecting specifics regions
df3 = df2[df2[regions_table].isin(regions_names)]
second_out_df = df3[months_list]

second_out_df = df3[months_list].astype(int)
second_out_df = second_out_df.transpose()

second_out_df


In [ ]:
# creates panda dataframe with specific sheets  - third_df

third_df = pd.read_excel(file_in, sheet_name=9, skiprows=4, header=[0, 1])


In [ ]:
# optimizes data mensal evolution by dates and sectors with Saldos

gr_act_econ_table = (
    'Grupamento de Atividades Econômicas e Seção CNAE 2.0', 'Unnamed: 1_level_1')
sector_names = ('Agricultura, pecuária, produção florestal, pesca e aquicultura', 'Indústria geral', 'Construção',
                'Comércio; reparação de veículos automotores e motocicletas', 'Serviços', 'Não identificado***')

# copying df for reutilizing
df2 = third_df.copy()

sectors_list = createColumnsList(df2)
df2 = cleanList(df2, sectors_list)


# selecting specifics sectors
df3 = df2[df2[gr_act_econ_table].isin(sector_names)]
third_out_df = df3[sectors_list]

third_out_df = df3[sectors_list].astype(int)
third_out_df = third_out_df.transpose()
third_out_df


In [ ]:
# creates csv file

first_columns_names_out = {'Mês': '', 'Saldos': 'total'}
second_columns_names_out = {1: 'reg1', 9: 'reg2',
                            19: 'reg3', 24: 'reg4', 28: 'reg5', 33: 'reg9'}
third_columns_names_out = {1: 'setA', 2: 'setE',
                           7: 'setF', 8: 'setG', 9: 'setU', 27: 'setZ'}

# remove index and columns
temp_first_out_df = first_out_df.reset_index(drop=True)
temp_first_out_df.rename(columns=first_columns_names_out, inplace=True)

temp_second_out_df = second_out_df.reset_index(drop=True)
temp_second_out_df.rename(columns=second_columns_names_out, inplace=True)

temp_third_out_df = third_out_df.reset_index(drop=True)
temp_third_out_df.rename(columns=third_columns_names_out, inplace=True)

final_out_df = pd.concat(
    [temp_first_out_df, temp_second_out_df, temp_third_out_df], axis=1)
final_out_df.to_csv(file_out, index=False)
final_out_df


In [ ]:
f_in = 'caged_2022_02.csv'
myf_in = 'caged.csv'

csv_df = pd.read_csv(f_in)
mycsv_df = pd.read_csv(myf_in)
csv_df.rename(columns={'Unnamed: 0': ''}, inplace=True)
mycsv_df.rename(columns={'Unnamed: 0': ''}, inplace=True)

#rdf = csv_df[list(mycsv_df.columns)]
# rdf
#mycsv_df == rdf
